# Teoria

### Do czego to potrzebne?

Wektorów i wartości własnych możemy użyć na przykład do redukcji wymiarowości, na ich podstawie oceniając istotność zmiennych

### Przekształcenia liniowe (linear transformation)

Endomorfizmem liniowym (rodzaj przekształcenia liniowego) jest funkcja $f(\textbf{w})$ operująca na wektorze kolumnowym długości $n$ postaci$$f(\textbf{w})=A\textbf{w},$$ gdzie $A$ jest macierzą kwadratową rozmiaru $n\times n$.

Dla $n=2$ endomorfizm liniowy jest zdefiniowany jako
$$f(\textbf{w})=A\textbf{w}=\begin{bmatrix}

a_{11} & a_{12} \\

a_{21} & a_{22}

\end{bmatrix}\begin{bmatrix}

w_1 \\

w_2

\end{bmatrix}.$$
Jeżeli każdy punkt na obrazie potraktujemy jako wektor $[x,y]$, możemy zobrazować jak działa przykładowy endomorfizm liniowy.


![](../img/2023-03-26-19-17-31.png)

![](../img/2023-03-26-19-21-14.png)

![](../img/2023-03-26-19-21-45.png)

### Wektory własne

Wektor własny wskazuje kierunek skalowania/rotacji jaki wykonuje [[Przekształcenie liniowe|przekształcenie liniowe]]
$$f(\textbf{w})=A\textbf{w},$$
Gdzie wzor powyżej opisuje konkretne przekształcenie (przemnożenie wektora W przez macierz przekształceń A)

![](../img/2023-03-26-19-24-22.png)

Natomiast **wartością własną** i związanym z nią **wektorem własnym** nazywamy wartość $\lambda$ i wektor $\mathbf{w}_0$ spełniające warunek$$f(\mathbf{w}_0)=\lambda \mathbf{w_0}$$Gdzie w0 opisuje wektor który określa kierunek w jakim zwrócony był wektor z któego wyznaczono wektor własny.
```
Wektor, który po przeskalowaniu (przemnożenie prez WARTOŚĆ WŁASNĄ) wskazuje ten sam kierunek co oryginalny wektor, jest jego wektorem własnym.

### W pythonie

In [1]:
import numpy as np
A = np.array([[ 1, 2, 3], [ 4, 5, 6], [ 7, 8, 22]])

wartosci_wlasne, wektory_wlasne = np.linalg.eig(A)

print(wartosci_wlasne) #eigen values - wartości własne
# [25.59639374 -0.52098034 2.9245866 ]

print(wektory_wlasne) #eigen vectors - wektory własne
#[[-0.13957085 -0.84384918 -0.21345016] 
# [-0.30183948 0.53152606 -0.87052838] 
# [-0.9430869 0.0734753 0.44341782]]

[25.59639374 -0.52098034  2.9245866 ]
[[-0.13957085 -0.84384918 -0.21345016]
 [-0.30183948  0.53152606 -0.87052838]
 [-0.9430869   0.0734753   0.44341782]]


### Więcej informacji:
- https://www.kowalskimateusz.pl/wektory-i-wartosci-wlasne-eigenvalues-and-eigenvectors/
- https://www.youtube.com/watch?v=qUeud6DvOWI (Artykul powyzej w wersji video)

# Przykładowe użycie w DataScience

Wykorzystanie do redukcji wymiarowości zbioru danych:

In [2]:
# 1. Wczytanie jakichkolwiek danych
import pandas as pd
csv_url = 'https://archive.ics.uci.edu/ml/machine-learning-databases/iris/iris.data'
col_names = ['Sepal_Length','Sepal_Width','Petal_Length','Petal_Width','Class']
df =  pd.read_csv(csv_url, names = col_names)
features = df[['Sepal_Length', 'Sepal_Width','Petal_Length','Petal_Width']]
targets = df['Class']
features

,Sepal_Length,Sepal_Width,Petal_Length,Petal_Width
0,5.1,3.5,1.4,0.2
1,4.9,3.0,1.4,0.2
2,4.7,3.2,1.3,0.2
3,4.6,3.1,1.5,0.2
4,5.0,3.6,1.4,0.2
...,...,...,...,...
145,6.7,3.0,5.2,2.3
146,6.3,2.5,5.0,1.9
147,6.5,3.0,5.2,2.0
148,6.2,3.4,5.4,2.3


In [3]:
# 2. Wyznaczamy macierz kowariancji (czyli zależność pomiędzy poszczególnymi kolumnami):
macierz_kowariancji = np.cov(features.T)
macierz_kowariancji

array([[ 0.68569351, -0.03926846,  1.27368233,  0.5169038 ],
       [-0.03926846,  0.18800403, -0.32171275, -0.11798121],
       [ 1.27368233, -0.32171275,  3.11317942,  1.29638747],
       [ 0.5169038 , -0.11798121,  1.29638747,  0.58241432]])

Kowariancja mierzy, jak bardzo dwie zmienne są ze sobą powiązane i jak bardzo jedna zmienna wpływa na wartość drugiej zmiennej.
Macierz kowariancji zawiera informacje o kowariancjach pomiędzy wszystkimi parami zmiennych w zbiorze danych.

W macierzy kowariancji:
- elementy po przekątnej odpowiadają wariancji każdej zmiennej, czyli mierzą jak bardzo wartości zmieniają się wokół swojego średniego poziomu. 
- Elementy poza przekątną to kowariancje pomiędzy parami zmiennych, czyli mierzą w jaki sposób zmiany jednej zmiennej są skorelowane ze zmianami drugiej zmiennej.

In [4]:
# Wyznaczamy wartości i wektory własne macierzy kowariancji:
wartosci_wlasne, wektory_wlasne = np.linalg.eig(macierz_kowariancji)
wartosci_wlasne, wektory_wlasne

(array([4.22484077, 0.24224357, 0.07852391, 0.02368303]),
 array([[ 0.36158968, -0.65653988, -0.58099728,  0.31725455],
        [-0.08226889, -0.72971237,  0.59641809, -0.32409435],
        [ 0.85657211,  0.1757674 ,  0.07252408, -0.47971899],
        [ 0.35884393,  0.07470647,  0.54906091,  0.75112056]]))

In [5]:
# Sortujemy wartości własne malejąco. 
# Te, które mają większe wartości, odpowiadają kierunkom w przestrzeni, w których zmienność danych jest większa (a więc zawierają najwięcej informacji)
indeksy_sort = wartosci_wlasne.argsort()[::-1] #argsort zwraca numery indexow od najmniejszej wartosci, [::-1] odwraca kolejnosc
wartosci_wlasne_posortowane = wartosci_wlasne[indeksy_sort]
wektory_wlasne_posortowane = wektory_wlasne[:, indeksy_sort]
wartosci_wlasne_posortowane, wektory_wlasne_posortowane

(array([4.22484077, 0.24224357, 0.07852391, 0.02368303]),
 array([[ 0.36158968, -0.65653988, -0.58099728,  0.31725455],
        [-0.08226889, -0.72971237,  0.59641809, -0.32409435],
        [ 0.85657211,  0.1757674 ,  0.07252408, -0.47971899],
        [ 0.35884393,  0.07470647,  0.54906091,  0.75112056]]))

In [6]:
# Wykorzystujemy wektory własne do redukcji wymiarowości. 
# Możemy wybrać kilka największych wartości własnych i odpowiadające im wektory własne, a następnie pomnożyć macierz danych przez te wektory własne. 
# Otrzymamy w ten sposób nową macierz danych z mniejszą ilością wymiarów
najwieksze_wartosci_wlasne = wartosci_wlasne_posortowane[:2]
najwieksze_wektory_wlasne = wektory_wlasne_posortowane[:, :2]
new_features = features.dot(najwieksze_wektory_wlasne)
new_features

,0,1
0,2.827136,-5.641331
1,2.795952,-5.145167
2,2.621524,-5.177378
3,2.764906,-5.003599
4,2.782750,-5.648648
...,...,...
145,7.455360,-5.502139
146,7.037007,-4.939703
147,7.275389,-5.393243
148,7.412972,-5.430600


W ten sposób uzyskujemy nową macierz danych, która jest mniejsza wymiarowo, a więc łatwiejsza do przetworzenia.

In [47]:
# Przykład szybkiej nauki regresjii logistycznej dla zewryfikowania, czy dane po redukcji wymiarowości dalej mają wartość opisującą
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

# Ten warning informuje, że dobrze byłoby normalizować dane dla osiągnięcia lepszych wyników. Na potrzeby prezentacji przykładu możemy to zignorować
from sklearn.exceptions import ConvergenceWarning
from warnings import simplefilter
simplefilter("ignore", category=ConvergenceWarning)

# Podział na zbiór treningowy i testowy
X_train, X_test, y_train, y_test = train_test_split(new_features, targets, test_size=0.2, random_state=101)

# Trenowanie modelu
model = LogisticRegression()
model.fit(X_train, y_train)

# Ocena dokładności modelu
y_pred = model.predict(X_test)
for id, row in enumerate(zip(y_test, y_pred)):
    print('target vs prediction: ', row)
    if id>10: break

score = model.score(X_test, y_test)
print("\nDokładność modelu po redukcji wymiarów:", score)

# # DLA PORÓWNANIA MODEL WYTRENOWANY NA ORYGINALNYCH DANYCH (BEZ REDUKCJI WYMIARÓW):
X_train, X_test, y_train, y_test = train_test_split(features, targets, test_size=0.2, random_state=101)
model = LogisticRegression()
model.fit(X_train, y_train)
score = model.score(X_test, y_test)
print("Dokładność modelu z oryginalnych danych:", score)

print("\nUwaga - wyniki modeli sa uzaleznione od random-state. \nDla niektórych wynik po redukcji będzie jednakowy albo nawet lepszy niż przed!")

target vs prediction:  ('Iris-setosa', 'Iris-setosa')
target vs prediction:  ('Iris-setosa', 'Iris-setosa')
target vs prediction:  ('Iris-setosa', 'Iris-setosa')
target vs prediction:  ('Iris-virginica', 'Iris-virginica')
target vs prediction:  ('Iris-versicolor', 'Iris-versicolor')
target vs prediction:  ('Iris-virginica', 'Iris-virginica')
target vs prediction:  ('Iris-versicolor', 'Iris-versicolor')
target vs prediction:  ('Iris-versicolor', 'Iris-versicolor')
target vs prediction:  ('Iris-virginica', 'Iris-virginica')
target vs prediction:  ('Iris-setosa', 'Iris-setosa')
target vs prediction:  ('Iris-virginica', 'Iris-virginica')
target vs prediction:  ('Iris-setosa', 'Iris-setosa')

Dokładność modelu po redukcji wymiarów: 0.9666666666666667
Dokładność modelu z oryginalnych danych: 1.0

Uwaga - wyniki modeli sa uzaleznione od random-state. 
Dla niektórych wynik po redukcji będzie jednakowy albo nawet lepszy niż przed!
